# Simulate PAR-network via deterministic PDE system

In [515]:
import bisect
import copy
import glob
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle
from scipy import optimize
from scipy.interpolate import interp1d
import sys
import time
# Profiling
import cProfile
import re
sys.path.append(os.path.abspath('../')+'/PythonDet/')
from ParDetAsymm import ParSim, Sim_Container, s_to_v
# from ParDetAsymmO import ParSim as ParSimO

In [ ]:
# For graphics settings modify matplotlibrc in /Users/hubatsl/anaconda3/pkgs/matplotlib-2.0.2-np111py35_0/lib/python3.5/site-packages/matplotlib/mpl-data


In [ ]:
# Define colors for plotting
c1 = (132/255, 197/255, 97/255) # green
c2 = (223/255, 184/255, 88/255) # yellow

In [ ]:
# Try out breakdown for wild-type parameters with PAR-2 diffusion
# adapting roughly according to scaling relationship of gradients

n = 8
wt = {'alpha': 1, 'beta': 2, 'dA': 0.23, 'dP': 0.0,
          'kAP': 0.19, 'kPA': 2, 'koffA': 0.0047,
          'koffP': 0.0054, 'konA': 0.00858, 'konP': 0.0474,
          'Ptot': 1, 'ratio': 1.56, 'sys_size': 134.6/2}
sizes = np.linspace(5, 30, n)
s_factor = ((0.058*sizes+4.739)/(0.058*134.6+4.739))**2
sizes = sizes/2 # Adjust to half system size simulation
siz = [copy.deepcopy(wt) for _ in sizes]
for i, dic in enumerate(siz):
    siz[i]['sys_size'] = sizes[i]
    siz[i]['dP'] = wt['dP']*s_factor[i]
    
s1 = Sim_Container(siz, no_workers=8)
s1.init_simus()
s1.run_simus()
s1.pickle_data('CPSS_adapt_to_gradient')

**Materials and methods plot thesis**

In [ ]:
wt16 = {'alpha': 1, 'beta': 2, 'dA': 0.31, 'dP': 0.3,
          'kAP': 0.19, 'kPA': 2, 'koffA': 0.0074,
          'koffP': 0.005, 'konA': 0.00858, 'konP': 0.0474,
          'Ptot': 1, 'ratio': 1.56, 'sys_size': 16}
wt18 = {'alpha': 1, 'beta': 2, 'dA': 0.31, 'dP': 0.3,
          'kAP': 0.19, 'kPA': 2, 'koffA': 0.0073,
          'koffP': 0.005, 'konA': 0.00858, 'konP': 0.0474,
          'Ptot': 1, 'ratio': 1.56, 'sys_size': 18}
wt20 = {'alpha': 1, 'beta': 2, 'dA': 0.3, 'dP': 0.31,
          'kAP': 0.19, 'kPA': 2, 'koffA': 0.0072,
          'koffP': 0.005, 'konA': 0.00858, 'konP': 0.0474,
          'Ptot': 1, 'ratio': 1.56, 'sys_size': 20}
wt22 = {'alpha': 1, 'beta': 2, 'dA': 0.3, 'dP': 0.3,
          'kAP': 0.19, 'kPA': 2, 'koffA': 0.0071,
          'koffP': 0.005, 'konA': 0.00858, 'konP': 0.0474,
          'Ptot': 1, 'ratio': 1.56, 'sys_size': 22}
wt30 = {'alpha': 1, 'beta': 2, 'dA': 0.29, 'dP': 0.29,
          'kAP': 0.19, 'kPA': 2, 'koffA': 0.007,
          'koffP': 0.005, 'konA': 0.00858, 'konP': 0.0474,
          'Ptot': 1, 'ratio': 1.56, 'sys_size': 30}
wt = {'alpha': 1, 'beta': 2, 'dA': 0.23, 'dP': 0.26,
          'kAP': 0.19, 'kPA': 2, 'koffA': 0.0047,
          'koffP': 0.0054, 'konA': 0.00858, 'konP': 0.0474,
          'Ptot': 1, 'ratio': 1.56, 'sys_size': 134.6/2}

In [ ]:
# s_wt16 = ParSim(param_dict=wt16, T=1000, grid_size=200)
# s_wt16.simulate()
# s_wt18 = ParSim(param_dict=wt18, T=1000, grid_size=200)
# s_wt18.simulate()
# s_wt20 = ParSim(param_dict=wt20, T=1000, grid_size=200)
# s_wt20.simulate()
# s_wt22 = ParSim(param_dict=wt22, T=1000, grid_size=200)
# s_wt22.simulate()
# s_wt30 = ParSim(param_dict=wt30, T=1000, grid_size=200)
# s_wt30.simulate()
# s_wt = ParSim(param_dict=wt, T=1000, grid_size=200)
# s_wt.simulate()

In [ ]:
s_wt16.plot_steady_state()
s_wt18.plot_steady_state()
s_wt20.plot_steady_state()
s_wt22.plot_steady_state()
s_wt30.plot_steady_state()
s_wt.plot_steady_state()

Write to csv to be read into Matlab

In [ ]:
def write_for_matlab(s, name):
    pd.DataFrame(s.A[:,-1]).to_csv('/Users/hubatsl/Desktop/interplay-cell-size/MatsMets/Anterior' + name + '.csv', header=False, index=False)
    pd.DataFrame(s.P[:,-1]).to_csv('/Users/hubatsl/Desktop/interplay-cell-size/MatsMets/Posterior' + name + '.csv', header=False, index=False)
write_for_matlab(s_wt16, '16')
write_for_matlab(s_wt18, '18')
write_for_matlab(s_wt20, '20')
write_for_matlab(s_wt22, '22')
write_for_matlab(s_wt30, '30')
write_for_matlab(s_wt, '67')

In [ ]:
li = []
dA_polarized_00001 = []
dP_polarized_00001 = []
dA_unpolarized_00001 = []
dP_unpolarized_00001 = []
dA_polarized_001 = []
dP_polarized_001 = []
dA_unpolarized_001 = []
dP_unpolarized_001 = []
for file in glob.glob("/Volumes/Transcend/pickled/*.pickle"):
#     print(file)
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 1) & (p.ss_prec == 1.00001):
        dA_unpolarized_00001.append(p.dA)
        dP_unpolarized_00001.append(p.dP)
    elif (p.finished_in_time == 2)& (p.ss_prec == 1.00001):
        dA_polarized_00001.append(p.dA)
        dP_polarized_00001.append(p.dP)
    if (p.finished_in_time == 1) & (p.ss_prec == 1.001):
        dA_unpolarized_001.append(p.dA)
        dP_unpolarized_001.append(p.dP)
    elif (p.finished_in_time == 2)& (p.ss_prec == 1.001):
        dA_polarized_001.append(p.dA)
        dP_polarized_001.append(p.dP)
        

In [ ]:
dA_polarized_fixTend_NateThresh = []
dP_polarized_fixTend_NateThresh = []
dA_unpolarized_fixTend_NateThresh = []
dP_unpolarized_fixTend_NateThresh = []
for file in glob.glob("/Users/hubatsl/Desktop/pickled_NateThresh/*.pickle"):
#     print(file)
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 1):
        dA_unpolarized_fixTend_NateThresh.append(p.dA)
        dP_unpolarized_fixTend_NateThresh.append(p.dP)
    elif (p.finished_in_time == 2):
        dA_polarized_fixTend_NateThresh.append(p.dA)
        dP_polarized_fixTend_NateThresh.append(p.dP)

In [ ]:
plt.plot(dA_polarized_fixTend_NateThresh, dP_polarized_fixTend_NateThresh,'.')
plt.plot(dA_unpolarized_fixTend_NateThresh, dP_unpolarized_fixTend_NateThresh,'.')
plt.show()

In [ ]:
dA_polarized_Time14400 = []
dP_polarized_Time14400 = []
dA_unpolarized_Time14400 = []
dP_unpolarized_Time14400 = []
for file in glob.glob("/Users/hubatsl/Desktop/pickled_Time14400/*.pickle"):
#     print(file)
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 1):
        dA_unpolarized_Time14400.append(p.dA)
        dP_unpolarized_Time14400.append(p.dP)
    elif (p.finished_in_time == 2):
        dA_polarized_Time14400.append(p.dA)
        dP_polarized_Time14400.append(p.dP)

In [ ]:
plt.plot(dA_polarized_Time14400, dP_polarized_Time14400,'.')
plt.plot(dA_unpolarized_Time14400, dP_unpolarized_Time14400,'.')
plt.show()

In [ ]:
dA_polarized_fixTend_Thresh03 = []
dP_polarized_fixTend_Thresh03 = []
dA_unpolarized_fixTend_Thresh03 = []
dP_unpolarized_fixTend_Thresh03 = []
for file in glob.glob("/Users/hubatsl/Desktop/pickled_fixTend_Thresh03/*.pickle"):
#     print(file)
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 1):
        dA_unpolarized_fixTend_Thresh03.append(p.dA)
        dP_unpolarized_fixTend_Thresh03.append(p.dP)
    elif (p.finished_in_time == 2):
        dA_polarized_fixTend_Thresh03.append(p.dA)
        dP_polarized_fixTend_Thresh03.append(p.dP)

In [ ]:
plt.plot(dA_polarized_fixTend_Thresh03, dP_polarized_fixTend_Thresh03,'.')
plt.plot(dA_unpolarized_fixTend_Thresh03, dP_unpolarized_fixTend_Thresh03,'.')
plt.show()

In [ ]:
dA_polarized_fixTend = []
dP_polarized_fixTend = []
dA_unpolarized_fixTend = []
dP_unpolarized_fixTend = []
for file in glob.glob("/Users/hubatsl/Desktop/pickled_fixTend/*.pickle"):
#     print(file)
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 1):
        dA_unpolarized_fixTend.append(p.dA)
        dP_unpolarized_fixTend.append(p.dP)
    elif (p.finished_in_time == 2):
        dA_polarized_fixTend.append(p.dA)
        dP_polarized_fixTend.append(p.dP)

In [ ]:
dA_polarized_fixTend_rerun = []
dP_polarized_fixTend_rerun = []
dA_unpolarized_fixTend_rerun = []
dP_unpolarized_fixTend_rerun = []
for file in glob.glob("/Users/hubatsl/Desktop/pickled_fixTend_rerun/*.pickle"):
#     print(file)
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 1):
        dA_unpolarized_fixTend_rerun.append(p.dA)
        dP_unpolarized_fixTend_rerun.append(p.dP)
    elif (p.finished_in_time == 2):
        dA_polarized_fixTend_rerun.append(p.dA)
        dP_polarized_fixTend_rerun.append(p.dP)

In [ ]:
plt.plot(dA_polarized_fixTend_rerun, dP_polarized_fixTend_rerun,'.')
plt.plot(dA_unpolarized_fixTend_rerun, dP_unpolarized_fixTend_rerun,'.')
plt.show()

In [ ]:
plt.plot(dA_polarized_fixTend, dP_polarized_fixTend,'.')
plt.plot(dA_unpolarized_fixTend, dP_unpolarized_fixTend,'.')
plt.show()

Threshold 0.05 or 0.3 doesn't seem to change much?? Verify file got recompiled!

In [ ]:
file = '/Users/hubatsl/Desktop/pickled_NateThresh/P_0.159493670886_dA_2.5.pickle'
with open(file, 'rb') as f:
    a = pickle.load(f)

In [ ]:
li = []
for file in glob.glob("/Users/hubatsl/Desktop/pickled_fixTend/*.pickle"):
    with open(file, 'rb') as f:
        mt = pickle.load(f)
        li.append(mt)

In [ ]:
dA_polarized = []
dP_polarized = []
dA_unpolarized = []
dP_unpolarized = []

for p in li:
    if p.finished_in_time == 1:
        dA_unpolarized.append(p.dA)
        dP_unpolarized.append(p.dP)
    elif p.finished_in_time == 2:
        dA_polarized.append(p.dA)
        dP_polarized.append(p.dP)

In [ ]:
# %matplotlib notebook
plt.plot(dA_unpolarized, dP_unpolarized, '.')
plt.plot(dA_polarized, dP_polarized, '.')
plt.xlim(-0.2, 3)
plt.ylim(-0.2, 3)
plt.axis('equal')
plt.show()

In [ ]:
in_vivo = {'alpha': 1, 'beta': 2, 'dA': 0.28, 'dP': 0.15,
           'kAP': 0.19, 'kPA': 2, 'koffA': 0.0054,
           'koffP': 0.0073, 'konA': 0.00858, 'konP': 0.0474,
           'Ptot': 1, 'ratio': 1.56, 'StoV': 0.174,
           'sys_size': 134.6/2}

## Diffusion versus steepest slope
### PAR system

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.01,
             'sys_size': 30}
n = 8
Ds = np.linspace(0.01, 1, 8)**2
siz = [copy.deepcopy(symmetric) for _ in Ds]
for i, dic in enumerate(siz):
    siz[i]['dA'] = Ds[i]
    siz[i]['dP'] = Ds[i]
s = Sim_Container(siz, no_workers=8)
s.init_simus()
s.run_simus()

In [ ]:
s.pickle_data('d_versus_sigma')

In [ ]:
# Thesis
def make_graph_pretty(xlab, ylab, loc=0, markerfirst=True, handlelength=None):
    color1 = 'black'#'darkorange'
    color2 = (0, 0, 0)
    color2 = (1, 1, 1)
    ax = plt.gca()
    ax.tick_params(axis='x', colors=color1, which='both')
    ax.tick_params(axis='y', colors=color1, which='both')
    plt.xlabel(xlab, color=color1)
    plt.ylabel(ylab, color=color1)
    ax.spines['left'].set_color(color1)
    ax.spines['bottom'].set_color(color1)
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_facecolor(color2)
    plt.gcf().subplots_adjust(bottom=0.17, left=0.15)
    plt.legend(frameon=False, loc=loc, labelspacing=0.1,
               markerfirst=markerfirst, handlelength=handlelength)
# Paper   
# def make_graph_pretty(xlab, ylab, loc=0, markerfirst=True, handlelength=None):
#     mpl.rcParams['lines.linewidth'] = 1
#     mpl.rcParams['lines.markersize']=4
#     color1 = 'black'#'darkorange'
#     color2 = (0, 0, 0)
#     color2 = (1, 1, 1)
#     ax = plt.gca()
#     ax.tick_params(axis='x', colors=color1, direction='in', which='both')
#     ax.tick_params(axis='y', colors=color1, direction='in', which='both')
#     plt.xlabel(xlab, color=color1)
#     plt.ylabel(ylab, color=color1)
#     ax.spines['left'].set_color(color1)
#     ax.spines['bottom'].set_color(color1)
#     # Hide the right and top spines
#     ax.spines['right'].set_visible(False)
#     ax.spines['top'].set_visible(False)
#     ax.set_facecolor(color2)
#     plt.gcf().subplots_adjust(bottom=0.29, left=0.27)
#     plt.legend(frameon=False, loc=loc, labelspacing=0.1, 
size=8,
#                markerfirst=markerfirst, handlelength=handlelength)
#     for tick in ax.get_xticklabels():
#         tick.set_fontname("Arial")
        
    
with open('d_versus_sigma.pickle', 'rb') as f:
    s1 = pickle.load(f) 
l = []
l1= []
l2 = []
l3 = []
D = []
for si in s1.simus:
    D.append(si.dA)
    i = np.argmax(np.sum(si.A, 0)==si.grid_size) - 1
    m_max = min(np.diff(si.A[:, i]))
    j = np.argmin(np.diff(si.A[:, i])-m_max)
    l.append(1/abs(m_max/(si.A[j, i])))

l = np.array(l[:-3])
D = np.array(D[:-3])

def fit_sqrt(D, A):
    return A*np.sqrt(D)
popt, pcov = optimize.curve_fit(fit_sqrt, D, l)
plt.figure(figsize=(0.6*3.75, 0.6*3.1))
plt.plot(D, l, 'o', lw=2, label='PAR system', color=c1)
domain = np.linspace(0, 0.35, 100)
plt.plot(domain, fit_sqrt(domain, *popt), '--', label='Square root fit', color=c2)
make_graph_pretty('$D\; [\mu m^2/s]$', '$\lambda [\mu m]$', loc=(0.1, 0.02))

# plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure2_Theory1/Slope_Vs_D1.pdf', transparent=True)
plt.show()

In [ ]:
for si in s.simus:
    D.append(si.dA)
    i = np.argmax(np.sum(si.A, 0)==si.grid_size) - 1
    m_max = min(np.diff(si.A[:, i]))
    j = np.argmin(np.diff(si.A[:, i])-m_max)
    l.append(1/abs(m_max/(si.A[j, i])))

### Wave pinning

In [ ]:
Da = pd.read_csv('/Users/hubatsl/Desktop/interplay-cell-size/Theory/Da.csv', index_col=False, header=None)
f  = pd.read_csv('/Users/hubatsl/Desktop/interplay-cell-size/Theory/fit.csv', index_col=False, header=None)
wp = pd.read_csv('/Users/hubatsl/Desktop/interplay-cell-size/Theory/wave_pin.csv', index_col=False, header=None)


plt.figure(figsize=(3.75, 3.1))
plt.plot(Da, 1/wp, 'o', lw=2, label='Wave Pinning', color=c1)
plt.plot(np.linspace(0, max(Da[0]), 100), f, '--', label='Best square root fit', color=c2)
make_graph_pretty(r'$D\; [\mu m^2/s]$', r'$Gradient\; Length\; [\mu m]$', loc=4)
plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/Slope_Vs_D_wavePin.pdf', transparent=True)
plt.show()


## How are D and $L_{crit}$ related?

In [ ]:
S = {'S': 134.6/2*np.logspace(-1, 1, 8)}
D = {'D': (S['S']/69.9)**2}
Simus = []
for d, size in zip(D['D'], S['S']):
    size_range = {'S': size*np.linspace(0.8, 1.2, 8)}
    Simu = Sim_Container(d, size_range)
    Simu.init_simus()
    Simu.run_simus()
    Simus.append(Simu)

In [ ]:
for s in Simus:
    s.pickle_data(str(s.simus[0].dx*s.simus[0].grid_size))

In [ ]:
s = [5, 10, 20, 39, 75, 144, 279, 538]
Simus = []
for i in s:
    with open('DvsLcrit/' + str(i) + '.pickle', 'rb') as f:
        Simus.append(pickle.load(f))

In [ ]:
d_break = []
s_break = []
for S in Simus:
    l = []
    L = []
    for si in S.simus:
        L.append(si.dx*si.grid_size)
        i = np.argmax(np.sum(si.A, 0)==si.grid_size) - 1
        di = np.diff(si.A[:, i])/si.dx
        m_max = min(di)
        j = np.argmin(di-m_max)
        l.append(abs(m_max/(si.A[j, i])))
    l = np.array(l)
    L = np.array(L)
    j = np.argmin(abs(np.diff(l)-max(np.diff(l))))
    s_break.append(L[j])
    d_break.append(S.simus[0].d)

# Restrict to reasonably small D/size
d_break = np.array(d_break[0:-3])
s_break = np.array(s_break[0:-3])

# CPSS vs D
plt.figure(figsize=(3.75, 3.1))
plt.plot(d_break, s_break, 'o', label='PAR system', color=c1)
def fit_sqrt(D, A):
    return A*np.sqrt(D)
popt, pcov = optimize.curve_fit(fit_sqrt, d_break, s_break)
domain = np.linspace(min(d_break), max(d_break), 40)
plt.plot(domain, fit_sqrt(domain, *popt), '--', label='Square root fit', color=c2)
make_graph_pretty(r'$D\;[\mu m ^2/s]$', r'$CPSS\; [\mu m]$', loc=4)
plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/Lcrit_vsD.pdf',
            transparent=True)
plt.show()

# CPSS vs sqrt(D)
plt.plot(np.sqrt(d_break), s_break, 'o')
p = np.polyfit(np.sqrt(d_break), s_break, 1)
plt.plot(np.sqrt(d_break), p[0]*np.sqrt(d_break), '--')
make_graph_pretty(r'$\sqrt{D}\;[\mu m/\sqrt{s}]$', r'$CPSS\; [\mu m]$')
plt.show()

# Ldiff vs critical system length
d_break=np.array(d_break)
s_break=np.array(s_break)
plt.plot(np.sqrt(d_break/0.005), s_break, 'o')
p = np.polyfit(np.sqrt(d_break/0.005), s_break, 1)
plt.plot(np.sqrt(d_break/0.005), p[0]*np.sqrt(d_break/0.005), '--')
make_graph_pretty(r'$L_{diff}\;[\mu m]$', r'$CPSS\; [\mu m]$')
plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/Lcrit_vsLdiff.pdf',
            transparent=True)
plt.show()

In [ ]:
plt.figure(figsize=(3.75, 3.1))
d_break = np.array([0.9, 3.6, 14.4, 57.6, 230.4])
s_break = np.array([10, 20, 40, 80, 160])
plt.plot(d_break, s_break, 'o', label='Wave Pinning', color=c1)
def fit_sqrt(D, A):
    return A*np.sqrt(D)
popt, pcov = optimize.curve_fit(fit_sqrt, d_break, s_break)
domain = np.linspace(min(d_break), max(d_break), 40)
plt.plot(domain, fit_sqrt(domain, *popt), '--', label='Best square root fit', color=c2)
make_graph_pretty(r'$D\;[\mu m ^2/s]$', r'$CPSS\; [\mu m]$', loc=4)

plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/Lcrit_vsD_WavePin.pdf',
            transparent=True)
plt.show()

### How does slope depend on off rate?

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.01,
             'sys_size': 30}
n = 8
off_rates = np.linspace(np.sqrt(0.0005), np.sqrt(0.05), n)**2
ant_rates = np.linspace(np.sqrt(0.0005)/0.005, np.sqrt(0.05)/0.005, n)**2
on_rates = symmetric['konA']*off_rates/symmetric['koffA']
siz = [copy.deepcopy(symmetric) for _ in on_rates]
for i, dic in enumerate(siz):
    siz[i]['konA'] = on_rates[i]
    siz[i]['konP'] = on_rates[i]
    siz[i]['koffA'] = off_rates[i]
    siz[i]['koffP'] = off_rates[i]
    siz[i]['kPA'] = ant_rates[i]
    siz[i]['kAP'] = ant_rates[i]
s = Sim_Container(siz, no_workers=8)
s.init_simus()
s.run_simus()

In [ ]:
s.pickle_data('boundary_vs_AntOff')

In [ ]:
with open('boundary_vs_AntOff.pickle', 'rb') as f:
    s = pickle.load(f)
koff = []
l = []
weird_int = []
weird_ind = []
dist = []
for si in s.simus:
    koff.append(si.koffA)
    i = np.argmax(np.sum(si.A, 0)==si.grid_size) - 1
    int08 = np.argmax(si.A[:, i]<0.8)
    int01 = np.argmax(si.A[:, i]<0.1)
    dist.append(int01-int08)
    m_max = min(np.diff(si.A[:, i]))
    j = np.argmin(np.diff(si.A[:, i])-m_max)
#     l.append(abs(m_max/(si.A[j, i])))
    weird_ind.append(j)
    weird_int.append(si.A[j, i])
    l.append(abs(m_max))

l = np.array(l[1:])
koff = np.array(koff[1:])
dist = np.array(dist[1:])

def fit_sqrt(koff, A):
    return A*np.sqrt(koff)
popt, pcov = optimize.curve_fit(fit_sqrt, koff, l)

plt.figure(figsize=(0.6*3.75, 0.6*3.1))
plt.plot(koff, 1/l,'o', label='PAR system', color=c1)
domain = np.linspace(min(koff), max(koff), 40)
plt.plot(domain, 1/fit_sqrt(domain, *popt), '--', label='Square root fit', color=c2)
make_graph_pretty('$k_{off}$ [1/s]', '$\lambda [\mu m]$', loc=(0.1, 0.7))
plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure2_Theory1/Slope_Vs_koff1.pdf', transparent=True)
plt.show()

**Now for WP model**

In [ ]:
delta = pd.read_csv('/Users/hubatsl/Desktop/interplay-cell-size/Theory/delta_off.csv', index_col=False, header=None)
f  = pd.read_csv('/Users/hubatsl/Desktop/interplay-cell-size/Theory/fit_off.csv', index_col=False, header=None)
wp = pd.read_csv('/Users/hubatsl/Desktop/interplay-cell-size/Theory/wave_pin_off.csv', index_col=False, header=None)

plt.figure(figsize=(3.75, 3.1))
plt.plot(delta[9:-6], 1/wp[9:-6], 'o', lw=2, label='Wave Pinning', color=c1)
plt.plot(delta[9:-6], 1/f[9:-6], '--', label='Best square root fit', color=c2)
make_graph_pretty(r'$\delta\; [1/s]$', r'$Gradient\; Length\; [\mu m]$')

plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/Slope_Vs_koff_wavePin.pdf', transparent=True)
plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/Slope_Vs_koff_wavePin.pdf', transparent=True)
plt.show()

**Now for Otsuji model**

In [ ]:
Ds = pd.read_csv('/Users/hubatsch/Desktop/PARmodelling/MatlabDet/ActivatorInhibitor/Ds.csv', index_col=False, header=None)
wp = pd.read_csv('/Users/hubatsch/Desktop/PARmodelling/MatlabDet/ActivatorInhibitor/size.csv', index_col=False, header=None)
def fit_sqrt(Ds, A):
    return A*np.sqrt(Ds)
popt, pcov = optimize.curve_fit(fit_sqrt, Ds.iloc[:,0], wp.iloc[:,0])
f_interp = interp1d(Ds.loc[:,0], wp.loc[:,0], kind='cubic')
x_new = np.linspace(Ds.iloc[0], Ds.iloc[-1], num=110, endpoint=True)
plt.figure(figsize=(3.75, 3.1))

plt.plot(Ds, wp, 'o', markersize=7, label='Mass conserved \nActivator Inhibitor', color=c1)
plt.plot(x_new, fit_sqrt(x_new, *popt), '--', lw=2, label='Square root fit', color=c2)
make_graph_pretty(r'$D\;[\mu m ^2/s]$', r'$CPSS\; [\mu m]$', loc=4)

plt.savefig('/Users/hubatsch/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure2Supp/Diffusion_CPSS_Otsuji.pdf', transparent=True)
plt.show()

### Initial conditions Membrane & cytoplasmic concentrations

In [ ]:
mem = 0.009*1/(0.005+0.174*0.009)
print('Membrane average concetration: ' + str(mem))
cyto = 1-0.174*mem
print('Cytoplasm average concetration: ' + str(cyto))

### Toms reproduction of Science paper

In [ ]:
s = Sim_Container({'S': [0]}, sys='Tom', no_workers=1)
s.init_simus()
s.run_simus()
s.simus[0].plot_steady_state()

**Now with Hill function for antagonism**

In [ ]:
s = Sim_Container({'S': [0]}, sys='TomHill', no_workers=1)
s.init_simus()
s.run_simus()
s.simus[0].plot_steady_state()

In [ ]:
s.simus[0].show_movie()

In [ ]:
fig, axs = plt.subplots(3,2)
l = []
color_list = plt.cm.Paired_r(np.linspace(0, 1, 10))
for i, (A, P) in enumerate(zip(OffA, OffP)):
    axs[0][0].set_title('Off rate')
    axs[0][0].plot(A[:,-1], color=color_list[i])
    axs[0][0].plot(P[:,-1], color=color_list[i])
for i, (A, P) in enumerate(zip(DiffA, DiffP)):
    axs[0][1].set_title('Diffusion rate')
    axs[0][1].plot(A[:,-1], color=color_list[i])
    axs[0][1].plot(P[:,-1], color=color_list[i])
for i, (A, P) in enumerate(zip(OnA, OnP)):
    axs[1][0].set_title('On rate')
    axs[1][0].plot(A[:,-1], color=color_list[i])
    axs[1][0].plot(P[:,-1], color=color_list[i])
for i, (A, P) in enumerate(zip(AntA, AntP)):
    axs[1][1].set_title('Antagonism rate')
    axs[1][1].plot(A[:,-1], color=color_list[i])
    axs[1][1].plot(P[:,-1], color=color_list[i])
for i, (A, P) in enumerate(zip(AntA, AntP)):
    axs[1][1].set_title('Ratio of proteins')
    axs[1][1].plot(A[:,-1], color=color_list[i])
    axs[1][1].plot(P[:,-1], color=color_list[i])
plt.show()

### Figure 1: important length scales:

**Gradient:** critical point at minimum slope before breakdown
                
**Panels:** 
 - Gradient with different D and different $k_{off}$. Show they have opposite effects
 - Effects of $k_{on}$, $k_{off}$, $D$, ratio and $k_{Ant}$ on the gradient.
    
**Domain size:** In combination with the gradient allows for polarity only at a certain ratio.
                At lower sizes, the system becomes more sensitive to changes in dosage
                
**Panels:**
 - Dosage changes domain size
 - Dosage vs gradient length tolerance of total system size, i.e. what percentage of the cell must be occupied by a domain in order to allow maintenance?
 - Diffusion length predicts breakdown reliably

### Illustrative plots to showcase effect of changes in D, $k_{off}$, $k_{AP}$ etc.

#### Changing D

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 30}
n = 8
ds = np.logspace(-2, 0, n)
ds = [0.01, 0.1, 0.3]
siz = [copy.deepcopy(symmetric) for _ in ds]
for i, dic in enumerate(siz):
    siz[i]['dP'] = ds[i]
    siz[i]['dA'] = ds[i]
s = Sim_Container(siz, no_workers=8)
s.init_simus()
s.run_simus()
s.pickle_data('DiffExample')

In [ ]:
s.simus[1].StoV

In [ ]:
with open('DiffExample.pickle', 'rb') as f:
    s = pickle.load(f)
plt.figure(figsize=(0.6*3.75, 0.6*3.1))
s.simus[2].plot_steady_state(c1='orangered', c2='royalblue', alpha=1.0, linestyle=':',
                              lab=str(ds[2]), printLast=True)
s.simus[0].plot_steady_state(c1='orangered', c2='royalblue', alpha=1.0, linestyle='--',
                             lab=str(ds[0]), printLast=True)
s.simus[1].plot_steady_state(c1='orangered', c2='royalblue', alpha=1.0, linestyle='-',
                             lab=str(ds[1]), printLast=True)

# # Paper:
# plt.gca().spines['bottom'].set_linewidth(1.5)
# plt.gca().spines['left'].set_linewidth(1.5)
make_graph_pretty('x [$\mu m$]','C [a.u.]', loc=4, handlelength=1.5)
# plt.xticks([], [])
# plt.yticks([], [])
# plt.title('Diffusion')
# plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure2_Theory1/D_example.pdf',
#             transparent=True)
plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure2_Theory1/D_example1.pdf',
            transparent=True)
# Thesis
# plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/D_example.pdf',
#             transparent=True)
plt.show()

#### Changing Dosage

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 120}
n = 8
# ds = np.linspace(0.55, 1/.55, n)
ds = np.linspace(0.8, 2.2, n)
# ds = np.array([0.5, 0.7, 1, 1/0.7, 1/0.5])
siz = [copy.deepcopy(symmetric) for _ in ds]
for i, dic in enumerate(siz):
    siz[i]['ratio'] = ds[i]
    siz[i]['Ptot'] = 1/ds[i]
s1 = Sim_Container(siz, no_workers=8)
s1.init_simus()
s1.run_simus()
s1.pickle_data('DosageVsBound')

In [ ]:
plt.figure(figsize=(3.75, 3.1))
for i in s1.simus:
    i.plot_steady_state(norm=True,printLast=True)
make_graph_pretty('x [$\mu m$]','C [a.u.]', loc=4, handlelength=0.75)
plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure3_Theory2/DosageVsBoundPosition1_norm.pdf',
            transparent=True)
plt.show()


In [ ]:
dosage = []
l = []
weird_int = []
weird_ind = []
dist = []
for si in s1.simus:
    dosage.append(si.ratio)
    i = np.argmax(np.sum(si.A, 0)==si.grid_size) - 1
    int08 = np.argmax(si.A[:, i]<0.8)
    int01 = np.argmax(si.A[:, i]<0.1)
    dist.append(int01-int08)
    m_max = min(np.diff(si.A[:, i]))
    j = np.argmin(np.diff(si.A[:, i])-m_max)
#     l.append(abs(m_max/(si.A[j, i])))
    weird_ind.append(j)
    weird_int.append(si.A[j, i])
    l.append(abs(m_max))

l = np.array(l[:])
dosage = np.array(dosage[:])
dist = np.array(dist[:])

def fit_sqrt(dosage, A):
    return A*np.sqrt(dosage)
popt, pcov = optimize.curve_fit(fit_sqrt, dosage, l)

plt.figure(figsize=(0.6*3.75, 0.6*3.1))
ax = plt.plot(dosage, 1/l,'o', color=c1)
plt.ylim(-0.0, 12)
plt.xlim(0.4, 1.7)
domain = np.linspace(min(dosage), max(dosage), 40)
# plt.plot(domain, 1/fit_sqrt(domain, *popt), '--', label='Square root fit', color=c2)
make_graph_pretty('A/P ratio', '$\lambda [\mu m]$', loc=(0.3, 0.7))
plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure3_Theory2/Slope_Vs_Dosage.pdf', transparent=True)
plt.show()

#### Changing dosage in the full model

In [ ]:
param_dict = {'alpha': 1, 'beta': 2, 'dA': 0.28, 'dP': 0.15,
              'kAP': 0.19, 'kPA': 2, 'koffA': 0.0054,
              'koffP': 0.0073, 'konA': 0.00858, 'konP': 0.0474,
              'Ptot': 1, 'ratio': 1.56, 'sys_size': 134.6/2}
n = 16
ds = np.linspace(0.3, 1, n)
siz = [copy.deepcopy(param_dict) for _ in ds]
for i, dic in enumerate(siz):
    siz[i]['Ptot'] = ds[i]
    siz[i]['ratio'] = param_dict['ratio']/ds[i]
s1 = Sim_Container(siz, no_workers=8)
s1.init_simus()
s1.run_simus()
s1.pickle_data('Dosage_Domain')

In [ ]:
with open('Dosage_Domain.pickle', 'rb') as f:
    s1 = pickle.load(f)
plt.figure(figsize=(0.6*3.75, 0.6*3.1))
for i in s1.simus:
    i.plot_steady_state(c1='salmon', c2='cornflowerblue',
                               alpha=1.0, lab='100', printLast=True)
plt.show()

In [ ]:
p = np.zeros((16, 63))
for i in range(len(s1.simus)):
    obj = s1.simus[i]
    for j in range(np.shape(obj.P)[1]):
        p[i, j] = 1-np.argmax(np.diff(obj.P[:,j]))/len(np.diff(obj.P[:,j]))

for i in range(0,63,3):
    plt.plot(ds, p[:, i])
    
plt.ylim(0.0, 0.5); plt.xlim(0, 1.2)
plt.xlabel('Dosage of P')
plt.ylabel('Relative position')
plt.savefig('/Users/hubatsl/Desktop/Pos_vs_Dosage.pdf', transparent=True)
plt.show()
np.savetxt('Pos_Vs_Dosage.csv', p, delimiter=',')

#### Changing Antagonism

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 30}
# n = 8
# ds = np.logspace(-1.0, 2, n)
ds = [0.1, 1, 10]
siz = [copy.deepcopy(symmetric) for _ in ds]
for i, dic in enumerate(siz):
    siz[i]['kAP'] = ds[i]
    siz[i]['kPA'] = ds[i]
s1 = Sim_Container(siz, no_workers=8)
s1.init_simus()
s1.run_simus()
# s1.pickle_data('AntExample')

In [ ]:
# with open('AntExample.pickle', 'rb') as f:
#     s1 = pickle.load(f)
plt.figure(figsize=(0.6*3.75, 0.6*3.1))
s1.simus[0].plot_steady_state(c1='orangered', c2='royalblue', linestyle=':',
                               alpha=1.0, lab=str(ds[0]), printLast=True)
s1.simus[1].plot_steady_state(c1='orangered', c2='royalblue', linestyle='-',
                              alpha=0.6, lab=str(ds[1]), printLast=True)
s1.simus[2].plot_steady_state(c1='orangered', c2='royalblue', linestyle='--',
                              alpha=1.0, lab=str(ds[2]), printLast=True)

# # Paper:
# plt.gca().spines['bottom'].set_linewidth(1.5)
# plt.gca().spines['left'].set_linewidth(1.5)
make_graph_pretty('x [$\mu m$]','C [a.u.]', loc=(0.6, 0.1), handlelength=1.5)
# plt.xticks([], [])
# plt.yticks([], [])
# plt.title('Antagonism')
plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure2_Theory1/Ant_example1.pdf',
            transparent=True)

# Thesis
# plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/Ant_example.pdf',
#             transparent=True)

plt.show()

In [ ]:
plt.figure(figsize=(3.75, 3.1))
s1.simus[-1].plot_steady_state(c1='salmon', c2='cornflowerblue',
                               alpha=1.0, lab=r'$k_{ant} = 100$', norm=True, printLast=True)
s1.simus[1].plot_steady_state(c1='salmon', c2='cornflowerblue',
                              alpha=0.6, lab=r'$k_{ant} = 0.3$', norm=True, printLast=True)
s1.simus[0].plot_steady_state(c1='salmon', c2='cornflowerblue',
                              alpha=0.3, lab=r'$k_{ant} = 0.1$', norm=True, printLast=True)
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc=(0.6, 0.1), handlelength=0.75)

# Thesis
# plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/Ant_example_norm.pdf',
#             transparent=True)

plt.show()

For paper: simulate 0.1, 1, 10, 100 kAP

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 30}
n = 4
ds = [0.1, 1, 10, 100]
siz = [copy.deepcopy(symmetric) for _ in ds]
for i, dic in enumerate(siz):
    siz[i]['kAP'] = ds[i]
    siz[i]['kPA'] = ds[i]
s1 = Sim_Container(siz, no_workers=4)
s1.init_simus()
s1.run_simus()
s1.pickle_data('AntExamplePaper')

In [ ]:
with open('AntExamplePaper.pickle', 'rb') as f:
    s1 = pickle.load(f)
plt.figure(figsize=(0.6*3.75, 0.6*3.1))
s1.simus[0].plot_steady_state(c1='salmon', c2='cornflowerblue',
                               alpha=0.1, lab='0.1', printLast=True)
s1.simus[1].plot_steady_state(c1='salmon', c2='cornflowerblue',
                              alpha=0.35, lab='1', printLast=True)
s1.simus[2].plot_steady_state(c1='salmon', c2='cornflowerblue',
                              alpha=0.75, lab='10', printLast=True)
s1.simus[3].plot_steady_state(c1='salmon', c2='cornflowerblue',
                              alpha=1, lab='100', printLast=True)

# # Paper:
# plt.gca().spines['bottom'].set_linewidth(1.5)
# plt.gca().spines['left'].set_linewidth(1.5)
make_graph_pretty('x [$\mu m$]','C [a.u.]', loc=(0.6, 0.1), handlelength=0.75)
# plt.xticks([], [])
# plt.yticks([], [])
# plt.title('Antagonism')
plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure2_Theory1/Ant_example.pdf',
            transparent=True)

# Thesis
# plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/Ant_example.pdf',
#             transparent=True)

plt.show()

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 30}
n = 16
ds = np.logspace(-1.0, 2, n)
siz = [copy.deepcopy(symmetric) for _ in ds]
for i, dic in enumerate(siz):
    siz[i]['kAP'] = ds[i]
    siz[i]['kPA'] = ds[i]
s1 = Sim_Container(siz, no_workers=8)
s1.init_simus()
s1.run_simus()
s1.pickle_data('AntVsL')

In [ ]:
with open('AntVsL.pickle', 'rb') as f:
    s1 = pickle.load(f)
kant = []
l = []
for si in s1.simus:
    kant.append(si.kAP)
    i = np.argmax(np.sum(si.A, 0)==si.grid_size) - 1
    m_max = min(np.diff(si.A[:, i]))
#     j = np.argmin(np.diff(si.A[:, i])-m_max)
#     l.append(abs(m_max/(si.A[j, i])))
    l.append(abs(m_max))

plt.figure(figsize=(0.6*3.75, 0.6*3.1))
plt.plot(kant, [1/x for x in l], label='Interp. simulations')
plt.semilogx([kant[0], kant[5], kant[10], kant[-1]],[ 1/l[0], 1/l[5], 1/l[10], 1/l[-1]], 'o', label='Simulated in D', color=c1)
plt.ylim(0, 100)
make_graph_pretty('$k_{AP}\; [\mu m^4/s]$','$\lambda$ [$\mu$m]', loc=0, handlelength=0.75)
plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure2_Theory1/Grad_vs_kAP1.pdf',
            transparent=True)

plt.show()

Antagonism gradient

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 30}
n = 8
ds = np.logspace(-1.0, 2, n)
siz = [copy.deepcopy(symmetric) for _ in ds]
for i, dic in enumerate(siz):
    siz[i]['kAP'] = ds[i]
    siz[i]['kPA'] = ds[i]
s1 = Sim_Container(siz, no_workers=8)
s1.init_simus()
s1.run_simus()
s1.pickle_data('AntExample')

### Changing off rate and antagonism simultaneously has the same effect as changing D

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 30}
n = 8
# off_rates = np.linspace(np.sqrt(0.0005), np.sqrt(0.05), n)**2
off_rates = np.array([0.001, 0.005, 0.05])
ant_rates = off_rates/0.005 # Ant in sym system is normally 1, now scaled with off rates
on_rates = symmetric['konA']*off_rates/symmetric['koffA']
# off_rates = off_rates[[-1, -7, -5]]
# ant_rates = ant_rates[[-1, -7, -5]]
# on_rates = on_rates[[-1, -7, -5]]
siz = [copy.deepcopy(symmetric) for _ in on_rates]
for i, dic in enumerate(siz):
    siz[i]['konA'] = on_rates[i]
    siz[i]['konP'] = on_rates[i]
    siz[i]['koffA'] = off_rates[i]
    siz[i]['koffP'] = off_rates[i]
    siz[i]['kPA'] = ant_rates[i]
    siz[i]['kAP'] = ant_rates[i]
s = Sim_Container(siz, no_workers=8)
s.init_simus()
s.run_simus()
# s.pickle_data('AntOff_example')

In [ ]:
# with open('AntOff_example.pickle', 'rb') as f:
#     s1 = pickle.load(f)
plt.figure(figsize=(0.6*3.75, 0.6*3.1))
s.simus[0].plot_steady_state(c1='orangered', c2='royalblue', linestyle=':',
                               alpha=1, lab=r'0.2', printLast=True)
s.simus[1].plot_steady_state(c1='orangered', c2='royalblue', 
                               alpha=0.6, lab=r'1', printLast=True)
s.simus[2].plot_steady_state(c1='orangered', c2='royalblue', linestyle='--',
                               alpha=1.0, lab=r'10', printLast=True)
make_graph_pretty('x [$\mu m$]','C [a.u.]', loc=4, handlelength=1.5)

# # Paper:
# plt.gca().spines['bottom'].set_linewidth(1.5)
# plt.gca().spines['left'].set_linewidth(1.5)
# plt.xticks([], [])
# plt.yticks([], [])
# plt.title('Antagonism/Off rate')
plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure2_Theory1/OffAnt_example1.pdf',
            transparent=True)

# Thesis
# plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/OffAnt_example.pdf',
#             transparent=True)

plt.show()

#### Changing off rates

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 30}
# off_rates = np.linspace(0.001, 0.01, 8)
off_rates = np.array([0.004, 0.006, 0.012])
siz = [copy.deepcopy(symmetric) for _ in off_rates]
for i, dic in enumerate(siz):
    siz[i]['koffA'] = off_rates[i]
    siz[i]['koffP'] = off_rates[i]
s = Sim_Container(siz, no_workers=3)
s.init_simus()
s.run_simus()
s.pickle_data('Off_example')

In [ ]:
s1.simus

In [ ]:
with open('Off_example.pickle', 'rb') as f:
    s1 = pickle.load(f)
plt.figure(figsize=(3.75, 3.1))
s1.simus[0].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.3, 
                              lab=r'$k_{off} = 0.004  \;\frac{\mu m}{s}$')
s1.simus[1].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.6, 
                              lab=r'$k_{off} = 0.006 \;\frac{\mu m}{s}$')
s1.simus[2].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$k_{off} = 0.012 \;\frac{\mu m}{s}$')

plt.gca().set_ylim([0, 1.6])
# plt.xticks([], [])
# plt.yticks([], [])
# plt.gca().spines['bottom'].set_linewidth(1.5)
# plt.gca().spines['left'].set_linewidth(1.5)
# plt.title('Antagonism/Off rate')
# plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/On_example.pdf',
#             transparent=True)

# Thesis
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc='upper center',
                  handlelength=0.75)
# plt.title('On rate')
plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/Off_example.pdf',
            transparent=True)

plt.show()

In [ ]:
plt.figure(figsize=(3.75, 3.1))
s1.simus[0].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.3, 
                              lab=r'$k_{off} = 0.004  \;\frac{\mu m}{s}$', norm=True)
s1.simus[1].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.6, 
                              lab=r'$k_{off} = 0.006 \;\frac{\mu m}{s}$', norm=True)
s1.simus[2].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$k_{off} = 0.012 \;\frac{\mu m}{s}$', norm=True)

plt.gca().set_ylim([0, 1.6])
# plt.xticks([], [])
# plt.yticks([], [])
# plt.gca().spines['bottom'].set_linewidth(1.5)
# plt.gca().spines['left'].set_linewidth(1.5)
# plt.title('Antagonism/Off rate')
# plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/On_example.pdf',
#             transparent=True)

# Thesis
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc='upper center',
                  handlelength=0.75)
# plt.title('On rate')
plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/Off_example_norm.pdf',
            transparent=True)

plt.show()

#### Changing on rates and scaling antagonism

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 30}
on_rates = np.array([0.004, 0.006, 0.008])
siz = [copy.deepcopy(symmetric) for _ in on_rates]
for i, dic in enumerate(siz):
    siz[i]['konA'] = on_rates[i]
    siz[i]['konP'] = on_rates[i]
s = Sim_Container(siz, no_workers=3)
s.init_simus()
s.run_simus()
s.pickle_data('On_example')

In [ ]:
with open('On_example.pickle', 'rb') as f:
    s1 = pickle.load(f)
plt.figure(figsize=(3.75, 3.1))
s1.simus[0].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.3, 
                              lab=r'$k_{on} = 0.004  \;\frac{\mu m}{s}$')
s1.simus[1].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.6, 
                              lab=r'$k_{on} = 0.006 \;\frac{\mu m}{s}$')
s1.simus[2].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$k_{on} = 0.008 \;\frac{\mu m}{s}$')
# plt.gca().set_ylim([0, 2.2])
# plt.xticks([], [])
# plt.yticks([], [])
# plt.gca().spines['bottom'].set_linewidth(1.5)
# plt.gca().spines['left'].set_linewidth(1.5)
# plt.title('Antagonism/Off rate')
# plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/On_example.pdf',
#             transparent=True)

# Thesis
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc=4,
                  handlelength=0.75)
# plt.title('On rate')
plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/On_example.pdf',
            transparent=True)

plt.show()

In [ ]:
plt.figure(figsize=(3.75, 3.1))
s1.simus[0].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.3, 
                              lab=r'$k_{on} = 0.004  \;\frac{\mu m}{s}$', norm=True)
s1.simus[1].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.6, 
                              lab=r'$k_{on} = 0.006 \;\frac{\mu m}{s}$', norm=True)
s1.simus[2].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$k_{on} = 0.008 \;\frac{\mu m}{s}$', norm=True)# Thesis
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc=4,
                  handlelength=0.75)
# plt.title('On rate')
plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/On_example_norm.pdf',
            transparent=True)
plt.show()

#### Changing on rates and scaling antagonism

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 30}
on_rates = np.array([0.004, 0.006, 0.008])
ant_rates = (symmetric['konA']/on_rates)**2*symmetric['kAP']
siz = [copy.deepcopy(symmetric) for _ in on_rates]
for i, dic in enumerate(siz):
    siz[i]['konA'] = on_rates[i]
    siz[i]['konP'] = on_rates[i]
    siz[i]['kAP'] = ant_rates[i]
    siz[i]['kPA'] = ant_rates[i]
s = Sim_Container(siz, no_workers=3)
s.init_simus()
s.run_simus()
s.pickle_data('OnAnt_example')

In [ ]:
with open('OnAnt_example.pickle', 'rb') as f:
    s1 = pickle.load(f)
plt.figure(figsize=(3.75, 3.1))
s1.simus[0].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.3, 
                              lab=r'$k_{on} = 0.004  \;\frac{\mu m}{s}$')
s1.simus[1].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.6, 
                              lab=r'$k_{on} = 0.006 \;\frac{\mu m}{s}$')
s1.simus[2].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$k_{on} = 0.008 \;\frac{\mu m}{s}$')
# plt.gca().set_ylim([0, 2.2])
# plt.xticks([], [])
# plt.yticks([], [])
# plt.gca().spines['bottom'].set_linewidth(1.5)
# plt.gca().spines['left'].set_linewidth(1.5)
# plt.title('Antagonism/Off rate')
# plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/On_example.pdf',
#             transparent=True)

# Thesis
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc=4,
                  handlelength=0.75)
# plt.title('On rate')
plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/OnAnt_example.pdf',
            transparent=True)

plt.show()

In [ ]:
plt.figure(figsize=(3.75, 3.1))
s1.simus[0].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.3, 
                              lab=r'$k_{on} = 0.004  \;\frac{\mu m}{s}$', norm=True)
s1.simus[1].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.6, 
                              lab=r'$k_{on} = 0.006 \;\frac{\mu m}{s}$', norm=True)
s1.simus[2].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$k_{on} = 0.008 \;\frac{\mu m}{s}$', norm=True)# Thesis
# plt.xlim(-5, 5)
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc=4,
                  handlelength=0.75)
# plt.title('On rate')
plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/OnAnt_example_norm.pdf',
            transparent=True)
plt.show()

#### Wave pinning profiles for different Ds

In [ ]:
profs = pd.read_csv('/Users/hubatsl/Desktop/interplay-cell-size/Theory/ints_wp.csv', index_col=False, header=None)
plt.figure(figsize=(3.75, 3.1))
plt.plot(np.linspace(0, 40, 500), profs.transpose()[0], label=r'0.01 $\frac{\mu m^2}{s}$', color = 'cornflowerblue',alpha=1)
plt.plot(np.linspace(0, 40, 500), profs.transpose()[4], label=r'0.52 $\frac{\mu m^2}{s}$', color = 'cornflowerblue',alpha=0.6)
plt.plot(np.linspace(0, 40, 500), profs.transpose()[6], label=r'3.73 $\frac{\mu m^2}{s}$', color = 'cornflowerblue',alpha=0.3)
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc=4)
# plt.title('Diffusion in Wave Pinning')
plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/Diffusion_example_WP.pdf',
            transparent=True)
plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/Diffusion_example_WP.pdf',
            transparent=True)
plt.show()

#### Otsuji profiles for different Ds

In [ ]:
profs = pd.read_csv('/Users/hubatsch/Desktop/PARmodelling/MatlabDet/ActivatorInhibitor/ints_otsuji_off.csv', index_col=False, header=None)
# mpl.rcParams['font.family'] = 'serif'
# mpl.rcParams['font.serif'] = ['Palatino']

plt.figure(figsize=(3.75, 3.1))
plt.plot(np.linspace(0, 10, 200), profs[0], label=r'0.1 $\frac{\mu m^2}{s}$', color = 'cornflowerblue',alpha=1, lw=2)
plt.plot(np.linspace(0, 10, 200), profs[1], label=r'0.9 $\frac{\mu m^2}{s}$', color = 'cornflowerblue',alpha=0.6, lw=2)
plt.plot(np.linspace(0, 10, 200), profs[2], label=r'3.6 $\frac{\mu m^2}{s}$', color = 'cornflowerblue',alpha=0.3, lw=2)
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc=4)
# plt.title('Diffusion in Wave Pinning')
# plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/Diffusion_example_WP.pdf',
#             transparent=True)
plt.savefig('/Users/hubatsch/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure2Supp/Diffusion_ex_Otsuji.pdf',
            transparent=True)
plt.show()

#### Wave pinning for different off rates, all rates are scaled in order to keep patterning the same except for D.

In [ ]:
profs = pd.read_csv('/Users/hubatsl/Desktop/interplay-cell-size/Theory/ints_wp_off.csv', index_col=False, header=None)
plt.figure(figsize=(3.75, 3.1))
plt.plot(np.linspace(0, 40, 500), profs.transpose()[10], label=r'0.33 [1/s]', color = 'cornflowerblue',alpha=0.3)
plt.plot(np.linspace(0, 40, 500), profs.transpose()[15], label=r'1.92 [1/s]', color = 'cornflowerblue',alpha=0.6)
plt.plot(np.linspace(0, 40, 500), profs.transpose()[23], label=r'31.62 [1/s]', color = 'cornflowerblue',alpha=1)
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc=4)
# plt.title('Off rate in Wave Pinning')
plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/Off_example_WP.pdf',
            transparent=True)
plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/Off_example_WP.pdf',
            transparent=True)
plt.show()

#### Different sizes, same plot
**Very important to keep S/V (psi) in all the simulations the same, which is not default behavior in ParDetAsymm**

In [ ]:
from ParDetAsymm import ParSim, Sim_Container, s_to_v
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 30}
sizes = np.array([7.5, 15, 30, 60, 120])
sizes = np.array([50, 100, 200])
siz = [copy.deepcopy(symmetric) for _ in sizes]
for i, dic in enumerate(siz):
    siz[i]['sys_size'] = sizes[i]
s = Sim_Container(siz, no_workers=3)
s.init_simus()
s.run_simus()
s.pickle_data('Size_example')

In [ ]:
with open('Size_example.pickle', 'rb') as f:
    s1 = pickle.load(f)
plt.figure(figsize=(3.75, 3.1))
s1.simus[0].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$L = 7.5  \;\mu m$', printLast=True)
s1.simus[1].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$L = 15  \;\mu m$', printLast=True)
s1.simus[2].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.8, 
                              lab=r'$L = 30\;\mu m$', printLast=True)
s1.simus[3].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.6, 
                              lab=r'$L = 60 \;\mu m$', printLast=True)
s1.simus[4].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.4, 
                              lab=r'$L = 120 \;\mu m$', printLast=True)

# Thesis
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc=4,
                  handlelength=0.75)
# plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/Size_example.pdf',
#             transparent=True)

plt.show()

In [ ]:
with open('Size_example.pickle', 'rb') as f:
    s1 = pickle.load(f)
plt.figure(figsize=(3.75, 2.1))
s1.simus[0].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.8, 
                              lab=r'$L = 50\;\mu m$', printLast=True)
s1.simus[1].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.6, 
                              lab=r'$L = 100 \;\mu m$', printLast=True)
s1.simus[2].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.4, 
                              lab=r'$L = 200 \;\mu m$', printLast=True)

# Paper
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc=4,
                  handlelength=0.75)
plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure2_Theory1/Size_example_small.pdf',
            transparent=True)

plt.show()

In [ ]:
with open('Size_example.pickle', 'rb') as f:
    s1 = pickle.load(f)
plt.figure(figsize=(3.75, 3.1))
s1.simus[1].plot_steady_state(c1='salmon', c2='royalblue',alpha=1, 
                              lab=r'$L = 15  \;\mu m$', linestyle='-', lw=5, printLast=True)
s1.simus[3].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$L = 60 \;\mu m$', linestyle='-',lw=4, printLast=True)
s1.simus[4].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$L = 120 \;\mu m$', linestyle='-', lw=2, printLast=True)

# Thesis
make_graph_pretty('x [$\mu m$]','C [a.u.]', loc=4,
                  handlelength=0.75)
# plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure2_Theory1/Size_exampleBig.pdf',
#             transparent=True)

plt.show()

To get prettier breakdown, also turn off check for steady state in ParDetAsymm, in order to not stop simulating once concentrations in the posterior or anterior are equal, but to go for the full time course

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.01,
             'sys_size': 30}
sizes = np.array([7.5, 15, 30, 60, 120])
siz = [copy.deepcopy(symmetric) for _ in sizes]
for i, dic in enumerate(siz):
    siz[i]['sys_size'] = sizes[i]
s = Sim_Container(siz, no_workers=3)
s.init_simus()
s.run_simus()
s.pickle_data('SizeRatio1_01_example')

In [ ]:
with open('SizeRatio1_01_example.pickle', 'rb') as f:
    s1 = pickle.load(f)
plt.figure(figsize=(3.75, 3.1))
s1.simus[0].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$L = 7.5  \;\mu m$')
s1.simus[1].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$L = 15  \;\mu m$')
s1.simus[2].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.8, 
                              lab=r'$L = 30\;\mu m$')
s1.simus[3].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.6, 
                              lab=r'$L = 60 \;\mu m$')
# s1.simus[4].plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.4, 
#                               lab=r'$L = 120 \;\mu m$')

# Thesis
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc=4,
                  handlelength=0.75)
# plt.title('On rate')
# plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/SizeRatio1_01_example.pdf',
#             transparent=True)

plt.show()

In [ ]:
with open('/Volumes/Transcend/PARmodellingData/pickled_FindA051217/ratio_0.989697265625size_39.4936708861.pickle', 'rb') as f:
    s1 = pickle.load(f)
with open('/Volumes/Transcend/PARmodellingData/pickled_FindA051217/ratio_0.9832031250000001size_39.4936708861.pickle', 'rb') as f:
    s2 = pickle.load(f)
with open('/Volumes/Transcend/PARmodellingData/pickled_FindA051217/ratio_0.9609375size_39.4936708861.pickle', 'rb') as f:
    s3 = pickle.load(f)
with open('/Volumes/Transcend/PARmodellingData/pickled_FindA051217/ratio_0.93125size_39.4936708861.pickle', 'rb') as f:
    s4 = pickle.load(f)

In [ ]:
plt.figure(figsize=(3.75, 3.1))
s1.plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=1, 
                              lab=r'$r = 0.99$')
s2.plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.8, 
                              lab=r'$r = 0.98$')
s3.plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.6, 
                              lab=r'$r = 0.96$')
s4.plot_steady_state(c1='salmon', c2='cornflowerblue',alpha=0.4, 
                              lab=r'$r = 0.93$')
make_graph_pretty('x [$\mu m$]','Concentration [a.u.]', loc=0,
                  handlelength=0.75)
plt.savefig('/Users/hubatsl/Desktop/interplay-cell-size/Theory/A_Pratio_example.pdf',
            transparent=True)
plt.show()

### Movie for viva presentation

In [ ]:
# Only P 
wt = {'alpha': 1, 'beta': 2, 'dA': 0.23, 'dP': 0.26/10,
          'kAP': 0, 'kPA': 0, 'koffA': 0.0047,
          'koffP': 0.0, 'konA': 0.00858/3, 'konP': 0.0,
          'Ptot': 1, 'ratio': 1.56, 'sys_size': 134.6/2/np.sqrt(10)}
# # Both species
# wt = {'alpha': 2, 'beta': 2, 'dA': 2.6, 'dP': 2.6,
#           'kAP': 1, 'kPA': 1, 'koffA': 0.0047,
#           'koffP': 0.0047, 'konA': 0.00858, 'konP': 0.00858,
#           'Ptot': 1, 'ratio': 0.8, 'sys_size': 134.6/2}
# # Breakdown
wt = {'alpha': 2, 'beta': 2, 'dA': 4.6, 'dP': 4.6,
          'kAP': 1, 'kPA': 1, 'koffA': 0.0047,
          'koffP': 0.0047, 'konA': 0.00858, 'konP': 0.00858,
          'Ptot': 1, 'ratio': 0.8, 'sys_size': 134.6/2}

s_wt = ParSim(param_dict=wt, T=500, grid_size=200, store_interval=1)
s_wt.simulate()

In [ ]:
s_wt.plot_steady_state()
plt.show()

In [ ]:
s_wt.save_movie(fname='breakdown')

### Movie for paper

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 0.98,
             'sys_size': 30}
n = 2
sizes = [7, 14]
siz = [copy.deepcopy(symmetric) for _ in sizes]
for i, dic in enumerate(siz):
    siz[i]['sys_size'] = sizes[i]
s1 = Sim_Container(siz, no_workers=8)
s1.init_simus()
s1.run_simus()

In [ ]:
s1.simus[0].save_movie(everynth=5)

In [ ]:
s1.simus[1].save_indiv_frames(everynth=1)

In [ ]:
A->M, p->c

## Python Wave Pinning

In [ ]:
ratio_above = 1.4
ratio_below = 0.9
ratio = 1.1
difference = 1
while difference > 0.01:
    s = ParSim(mechanism='WP', T=500, param_dict={'ratio': ratio,'sys_size': 134.6/2})
    s.simulate()
    if (np.max(s.A[:, -1])-np.min(s.A[:, -1]))/np.max(s.A[:-1]) > 0.1: # System polarized
        ratio_below = ratio
        ratio = (ratio_above - ratio)/2 + ratio
    else:
        ratio_above = ratio
        ratio = ratio - (ratio-ratio_below)/2
    difference = abs(ratio_old - ratio)
    print(ratio)
            

In [513]:
from multiprocessing import Pool
from ParDetAsymm import ParSim, sim_indiv
sizes = np.linspace(20, 40, 8)

def find_outline(location, ratio, ratio_above, ratio_below, sizes, T, thresh):
    difference = np.ones(len(sizes))
#     while (difference > thresh).all():
    simList = [ParSim(mechanism='WP', T=T, param_dict={'ratio': ratio,'sys_size': s}) for s in sizes]
    for sim in simList:
        sim.set_init_profile_wave_pin()
    pool = Pool(processes=8)
    res = pool.map(sim_indiv, simList)
    pool.close()
    pool.join()
#         ratio_old = ratio
#         for i, s in enumerate(res):
#             if (location is 'above') & (np.max(s.A[:, -1])-np.min(s.A[:, -1]))/np.max(s.A[:-1] > 0.1):
#                 ratio_below[i] = ratio[i]
#                 ratio[i] = (ratio_above[i] - ratio[i])/2 + ratio[i]
#             elif location is 'above':
#                 ratio_above[i] = ratio[i]
#                 ratio[i] = ratio[i] - (ratio[i]-ratio_below[i])/2
#             elif (location is 'below') & (np.max(s.A[:, -1])-np.min(s.A[:, -1]))/np.max(s.A[:-1] > 0.1):
#                 ratio_above[i] = ratio[i]
#                 ratio[i] = ratio[i] - (ratio[i]-ratio_below[i])/2
#             elif location is 'below':
#                 ratio_below[i] = ratio[i]
#                 ratio[i] = (ratio_above[i] - ratio[i])/2 + ratio[i]
#             difference[i] = abs(ratio_old[i] - ratio[i])
#         print(ratio)
#     return ratio_above, ratio_below

In [458]:
s = ParSim(mechanism='WP', T=500, param_dict={'ratio': 1.1375,'sys_size': 134.6/2})
s.simulate()


In [ ]:
i = 6
plt.plot(s.A[:, :])
# plt.plot(s.P[:, i])
plt.legend(['A'])
print(s.total-sum(s.A[:, -1]))
# plt.figure()
# plt.plot(s.P[:, :5])
# # plt.plot(s.P[:, i])
# plt.legend(['P'])